In [ ]:
import os
import sys
import bson
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from copy import deepcopy
from matplotlib import pyplot as plt
from IPython.display import clear_output
from matplotlib.colors import TABLEAU_COLORS

sys.path.append('../../..')
from uncertain.utils.data import Data
from uncertain.utils.training import train, run_study, load
from uncertain.utils.evaluation import test_vanilla, test_uncertain, test_chap_five

from uncertain.implicit.base import MF, MLP
from uncertain.implicit.extras import HeuristicUncertainty
from uncertain.implicit.DoubleMF import JointDoubleMF, SequentialDoubleMF
from uncertain.implicit.extras import HeuristicUncertainty
from uncertain.implicit.neural import GaussianMLP, MCDropout, Ensemble
from uncertain.implicit.bayesianMLP import BayesianMLP

if os.path.isfile('data.pkl'):
    with open('data.pkl', 'rb') as f:
        data = pickle.load(f)
    print(f'Printerest data loaded: {data.n_user} users, {data.n_item} items.')
    print(f'{len(data.train)} train, {len(data.val)} validation and {len(data.test)} test interactions.')
else:
    a = pd.read_csv('pinterest-20.train.rating.txt', sep='\t', header=None)
    b = pd.read_csv('pinterest-20.test.rating.txt', sep='\t', header=None)
    data = pd.concat([a, b], ignore_index=True).drop(columns=[2, 3])
    data.columns = ['user', 'item']
    data = Data(data, implicit=True, distances=False)
    with open('data.pkl', 'wb') as f:
        pickle.dump(data, f, protocol=4)
        
data.item_support[data.item_support == 0] = 1
data.item_support = data.item_support.astype(float)
data.user_support = data.user_support.astype(float)

base_batch_size = 1024 *10
trials = 0 ## 0 for eval only mode
patience = 2 ## Number of validation checks before ending training

In [ ]:
# Calculate the tail items accordind to AUR paper
import torch

n_inter = data.item_support.sum()
order = np.argsort(data.item_support)

for n_tail in range(data.n_item):
    ratio_tail = np.sum(data.item_support[order[:n_tail]]) / n_inter
    if ratio_tail > 0.5:
        print(n_tail)
        break

data.tail_items = torch.tensor(order[:n_tail])
data.head_items = torch.tensor(order[n_tail:])

# Baselines

## MF

In [ ]:
name = 'MF-BCE'
def init_model(**kwargs):
    return MF(data.n_user, data.n_item, embedding_dim=128, loss='BCE', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-5, 1e-2, log=True),
              'n_negatives': trial.suggest_int('neg', 1, 50)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=trials)
best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
mfbce = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])

results = test_vanilla(mfbce, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs

In [ ]:
name = 'MF-BPR'
def init_model(**kwargs):
    return MF(data.n_user, data.n_item, embedding_dim=128, loss='BPR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-5, 1e-2, log=True)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / 2)
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=trials)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])

results = test_vanilla(model, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs

In [ ]:
name = 'MF-MSE'
def init_model(**kwargs):
    return MF(data.n_user, data.n_item, embedding_dim=128, loss='MSE', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-5, 1e-2, log=True),
              'n_negatives': trial.suggest_int('neg', 1, 50)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=0)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
mfmse = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])

results = test_vanilla(mfmse, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs

## MLP

In [ ]:
name = 'MLP-BCE'
def init_model(**kwargs):
    return MLP(data.n_user, data.n_item, embedding_dim=128, loss='BCE', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'dropout': trial.suggest_float('dropout', 0, 0.2),
              'n_negatives': trial.suggest_int('neg', 1, 50)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=0)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
mlpbce = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
'''
results = test_vanilla(mlpbce, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs
'''

In [ ]:
name = 'MLP-BPR'
def init_model(**kwargs):
    return MLP(data.n_user, data.n_item, embedding_dim=128, loss='BPR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'dropout': trial.suggest_float('dropout', 0, 0.2)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / 2)
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=trials)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])

results = test_vanilla(model, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs

In [ ]:
name = 'MLP-MSE'
def init_model(**kwargs):
    return MLP(data.n_user, data.n_item, loss='MSE', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'dropout': trial.suggest_float('dropout', 0, 0.2),
              'n_negatives': trial.suggest_int('neg', 1, 50)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP
    
study = run_study(name, objective, n_trials=0)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])

results = test_vanilla(model, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs

## Results

In [ ]:
# Load results
results = {}
for key in os.listdir('results'):
    results[key.replace('.pkl', '').replace('_', ' ')] = pickle.load(open(os.path.join('results', key), 'rb'))
order = ['MF-MSE', 'MF-BCE', 'MF-BPR', 'MLP-MSE', 'MLP-BCE', 'MLP-BPR']
results = pd.DataFrame([results[key] for key in order], index=order)

# Plot aestetics
colors = [c for c in list(TABLEAU_COLORS)]
colors = {k:c for k, c in zip(results.index, colors)}
lines = ['o', 'v', '^', '<', '>', 's', 'p', '+', 'x', '*', 'p']
lines = {k: '-' + l for k, l, in zip(results.index, lines)}

# Results
print(results['FCP'])

## Top-K accuracy metrics
f, ax = plt.subplots(figsize=(10, 5), ncols=2)
for index, row in results.iterrows():
    ax[0].plot(np.arange(1, 11), row['MAP'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
    ax[1].plot(np.arange(1, 11), row['Recall'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax[0].set_xticks(np.arange(1, 11))
ax[0].set_xlabel('n', fontsize=20)
ax[0].set_ylabel('MAP@n', fontsize=20)
ax[1].set_xticks(np.arange(1, 11))
ax[1].set_xlabel('n', fontsize=20)
ax[1].set_ylabel('Recall@n', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.83, 1.15))
f.tight_layout()
f.savefig('plots/baselines.pdf', bbox_inches='tight')

In [ ]:
results

# Chapter 4

## User / Item support

In [ ]:
results = test_uncertain(HeuristicUncertainty(baseline=mfmse, user_uncertainty=-data.user_support), data, name='NUS', max_k=10)
print(results['MAP-Uncertainty'])

In [ ]:
results = test_uncertain(HeuristicUncertainty(baseline=mfmse, item_uncertainty=-data.item_support), data, name='NIS', max_k=10)
print(results['MAP-Uncertainty'])

## Ensemble

In [ ]:
# Load params
best_params = study.best_params
data.batch_size = int(base_batch_size / (best_params['neg'] + 1))
best_params = {'n_negatives': best_params['neg'], 'weight_decay': best_params['wd']}

# Train
if trials > 0:
    for i in range(4):
        model = init_model(**best_params)
        train(model, data, path='checkpoints/ensemble_MF', name=f'{i}')

# Load ensemble models
models = [mfmse]
for file in os.listdir('checkpoints/ensemble_MF'):
    models.append(init_model())
    models[-1] = models[-1].load_from_checkpoint(os.path.join('checkpoints/ensemble_MF', file))
    models[-1].eval()
ensemble = Ensemble(models)
clear_output(wait=True)
results = test_uncertain(ensemble, data, name='MF-ENSEMBLE', max_k=10)
print(results['MAP'])

## Network uncertainty

In [ ]:
## MC Dropout
dropout = MCDropout(base_model=mlpbce, mc_iteration=5)
results = test_uncertain(dropout, data, max_k=10, name='MCDropout')
print(results['MAP-Uncertainty'])

In [ ]:
results['Cuts']

In [ ]:
# BayesianNN
name = 'BayesianMLP'
def init_model(**kwargs):
    return BayesianMLP(data.n_user, data.n_item, embedding_dim=128, **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
              'n_negatives': trial.suggest_int('neg', 1, 20),
              'sample_train': trial.suggest_int('train_samples', 1, 10),
              'prior_pi': trial.suggest_categorical('pi', [1/4, 1/2, 3/4]),
              'prior_sigma_1': trial.suggest_categorical('sigma1', np.exp(-np.array([0, 1, 2]))),
              'prior_sigma_2': trial.suggest_categorical('sigma2', np.exp(-np.array([6, 7, 8])))}
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    params['num_batches'] = int(len(data.train) / data.batch_size)
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    print(params, data.batch_size)
    
    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=1)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])

results = test_vanilla(model, data, max_k=10, name=name)
clear_output(wait=True)
print(results['MAP'])
best_runs

In [ ]:
## Deep Ensemble
# Load params
best_params = study.best_params
data.batch_size = int(base_batch_size / (best_params['neg'] + 1))
best_params = {'n_negatives': best_params['neg'], 'dropout': best_params['dropout']}

# Train
if trials > 0:
    for i in range(4):
        model = init_model(**best_params)
        train(model, data, path='checkpoints/ensemble_MLP', name=f'{i}')

# Load ensemble models
models = [mlpbce]
for file in os.listdir('checkpoints/ensemble_MLP'):
    models.append(MLP(data.n_user, data.n_item))
    models[-1] = models[-1].load_from_checkpoint(os.path.join('checkpoints/ensemble_MLP', file))
    models[-1].eval()
ensemble = Ensemble(models)
clear_output(wait=True)
results = test_uncertain(ensemble, data, name='MLP-ENSEMBLE', max_k=10)
print(results['MAP-Uncertainty'])

In [ ]:
results['MF-ENSEMBLE'].pop('norm_unc')
results['MF-ENSEMBLE'].keys(), results['MF-ENSEMBLE']

## Results

In [ ]:
# Baseline results
results = {}
for key in os.listdir('results'):
    results[key.replace('.pkl', '').replace('_', ' ')] = pickle.load(open(os.path.join('results', key), 'rb'))
order = ['MF-MSE', 'MF-BCE', 'MF-BPR', 'MLP-MSE', 'MLP-BCE', 'MLP-BPR']
results = pd.DataFrame([results[key] for key in order], index=order)

# Plot aestetics
colors = [c for c in list(TABLEAU_COLORS)]
colors = {k:c for k, c in zip(results.index, colors)}
lines = ['o', 'v', '^', '<', '>', 's', 'p', '+', 'x', '*', 'p']
lines = {k: '-' + l for k, l, in zip(results.index, lines)}

## Baselines
f, ax = plt.subplots(figsize=(10, 5), ncols=2)
for index, row in results.iterrows():
    ax[0].plot(np.arange(1, 11), row['MAP'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
    ax[1].plot(np.arange(1, 11), row['Recall'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax[0].set_xticks(np.arange(1, 11))
ax[0].set_xlabel('n', fontsize=20)
ax[0].set_ylabel('MAP@n', fontsize=20)
ax[1].set_xticks(np.arange(1, 11))
ax[1].set_xlabel('n', fontsize=20)
ax[1].set_ylabel('Recall@n', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.83, 1.15))
f.tight_layout()
f.savefig('plots/baselines4.pdf', bbox_inches='tight')

# Uncertainty models results
results = {}
for key in os.listdir('results'):
    results[key.replace('.pkl', '').replace('_', ' ')] = pickle.load(open(os.path.join('results', key), 'rb'))
order = ['MF-BCE', 'MLP-MSE', 'NIS', 'MF-ENSEMBLE', 'MLP-ENSEMBLE', 'MCDropout', 'BayesianMLP']
results = pd.DataFrame([results[key] for key in order], index=order)
results.rename(index={'NIS': 'NEG-ITEM-SUPPORT', 'MF-BCE': 'MF', 'MLP-MSE': 'MLP'}, inplace=True)

# Plot aestetics
colors = [c for c in list(TABLEAU_COLORS)]
colors = {k:c for k, c in zip(results.index, colors)}
lines = ['o', 'v', '^', '<', '>', 's', 'p', '+', 'x', '*', 'p']
lines = {k: '-' + l for k, l, in zip(results.index, lines)}

# Correlation plot
f, ax = plt.subplots(figsize=(8, 5), ncols=1)
corr = results[['corr_usup', 'corr_isup']].drop(index=['MF', 'MLP'])
corr.columns=[r'#$R_{u\cdot}$', r'#$R_{\cdot i}$']
corr.loc['NEG-ITEM-SUPPORT', r'#$R_{\cdot i}$'] = np.nan
sns.heatmap(corr.round(3), annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')
plt.yticks(fontsize=15)
plt.xticks(rotation=45, fontsize=20)
plt.tight_layout()
plt.savefig('plots/corr4.pdf')

## Top-K accuracy metrics
f, ax = plt.subplots(figsize=(10, 5), ncols=2)
for index, row in results.iterrows():
    if index != 'NEG-ITEM-SUPPORT':
        ax[0].plot(np.arange(1, 11), row['MAP'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.arange(1, 11), row['Recall'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax[0].set_xticks(np.arange(1, 11))
ax[0].set_xlabel('n', fontsize=20)
ax[0].set_ylabel('MAP@n', fontsize=20)
ax[1].set_xticks(np.arange(1, 11))
ax[1].set_xlabel('n', fontsize=20)
ax[1].set_ylabel('Recall@n', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.86, 1.15))
f.tight_layout()
f.savefig('plots/accuracy4.pdf', bbox_inches='tight')

# MAP vs Uncertainty
results = results.drop(index=['MF', 'MLP'])
f, ax = plt.subplots(figsize=(8, 5))
x = np.arange(10) + 1
for index, row in results.iterrows():
    ax.plot(x, row['MAP-Uncertainty'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax.set(xticks = x)
ax.set_xlabel('Uncertainty bin', fontsize=25)
ax.set_ylabel('MAP@10', fontsize=20)
handles, labels = ax.get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(1.038, 1.15))
f.tight_layout()
f.savefig('plots/MAP-Uncertainty.pdf', bbox_inches='tight')

# Cuts
indexs = [index for index in order if index != 'Baseline']
f, ax = plt.subplots(nrows=3, ncols=1, figsize=(5, 10))
for index, row in results.iterrows():
    ax[0].plot(row['Cuts']['MAP'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
    ax[1].plot(1/(1 + np.exp(-row['Cuts']['Relevance'])), lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
    ax[2].plot(row['Cuts']['Coverage'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax[0].set_xticks(range(5))
ax[0].set_xticklabels(np.linspace(1, 0.2, 5).round(2))
ax[0].set_xlabel('Uncertainty quantile cut', fontsize=20)
ax[0].set_ylabel('MAP@10', fontsize=20)
ax[1].set_xticks(range(5))
ax[1].set_xticklabels(np.linspace(1, 0.2, 5).round(2))
ax[1].set_xlabel('Uncertainty quantile cut', fontsize=20)
ax[1].set_ylabel('Mean Predicted Relevance@10', fontsize=20)
ax[2].set_xticks(range(5))
ax[2].set_xticklabels(np.linspace(1, 0.2, 5).round(2))
ax[2].set_xlabel('Uncertainty quantile cut', fontsize=20)
ax[2].set_ylabel('Coverage', fontsize=20)
# handles, labels = ax[0].get_legend_handles_labels()
# f.legend(handles, labels, fontsize=15, bbox_to_anchor=(1.3, 1.1), ncol=5)
f.tight_layout()
f.savefig('plots/cuts4.pdf', bbox_inches="tight")

# UAC / URI
results[['UAC', 'URI']]

# Chapter 5

## DoubleMF

In [ ]:
name = 'AUR_final'
def init_model(**kwargs):
    return JointDoubleMF(data.n_user, data.n_item, embedding_dim=128, embedding_dim_var=128, beta=1e-2, ratio=0.9, loss='AUR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-5, 1e-3, log=True),
              'beta': trial.suggest_float('beta', 1e-3, 1, log=True),
              'gamma': trial.suggest_float('gamma', 1e-4, 1e-1, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-5, 1e-3),
              'n_negatives': trial.suggest_int('neg', 1, 30)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)

    # Rename model file
    os.rename('checkpoints/' + name + '/last.ckpt', 'checkpoints/' + name + '/' + params_str + '_last.ckpt')
    
    return MAP

study = run_study(name, objective, n_trials=0)
best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
aur = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
# results = test_chap_five(aur, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

## ICPMF

In [ ]:
name = 'CPMF_final'
def init_model(**kwargs):
    return JointDoubleMF(data.n_user, data.n_item, embedding_dim=128, embedding_dim_var=1, beta=1/2, ratio=0.9, loss='AUR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-5, 1e-4, log=True),
              'gamma': trial.suggest_float('gamma', 1e-4, 1e-1, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-6, 1e-4),
              'n_negatives': trial.suggest_int('neg', 1, 30)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    print(MAP, path, train_likelihood)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)

    # Rename model file
    os.rename('checkpoints/' + name + '/last.ckpt', 'checkpoints/' + name + '/' + params_str + '_last.ckpt')
    
    return MAP

study = run_study(name, objective, n_trials=0)
# best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
# model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
icpmf = init_model().load_from_checkpoint('/home/vcoscrato/Documents/RecSys/MF/tests/Pointwise/Pinterest/checkpoints/CPMF_final/lr=2.2180734217155113e-05-gamma=0.0012755765912784212-weight_decay=6.320190751204811e-05-n_negatives=19-epoch=69-val_MAP=0.08589229732751846.ckpt')
# results = test_chap_five(icpmf, data, max_k=10, name=name)

clear_output(wait=True)
# best_runs

## Neural

In [ ]:
name = 'Neural-AUR_final'
def init_model(**kwargs):
    return GaussianMLP(data.n_user, data.n_item, embedding_dim=128, beta=1e-2, ratio=0.9, loss='AUR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-5, 1e-3, log=True),
              'beta': trial.suggest_float('beta', 1e-3, 1, log=True),
              'gamma': trial.suggest_float('gamma', 1e-4, 1e-1, log=True),
              'dropout': trial.suggest_float('dropout', 0, 0.2),
              'n_negatives': trial.suggest_int('neg', 1, 30)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=trials)
best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
gaussianMLP = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
gaussianMLP.dropl.eval()
# results = test_chap_five(gaussianMLP, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

## Hierarchical Bayesian Ranking (HBR)

In [ ]:
from uncertain.implicit.pyro import HBR, train, visualize

hbr = HBR(data.n_user, data.n_item, embedding_dim=128)
visualize(hbr, data)

In [ ]:
name = 'HBR_final'
def init_model(**kwargs):
    return HBR(data.n_user, data.n_item, embedding_dim=128, **kwargs)

def objective(trial):
    pyro.clear_param_store()
    
    # Parameter setup
    tau_ui = trial.suggest_float('tau_gamma', 1e-5, 1e-1, log=True)
    params = {'lr': trial.suggest_float('lr', 1e-5, 1e-3, log=True),
              'tau_gamma': trial.suggest_float('tau_gamma', 1e-5, 1e-1, log=True),
              'tau_u': tau_ui,
              'tau_i': tau_ui,
              'n_negatives': trial.suggest_int('neg', 1, 30)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP = train(model, data, n_steps=200, val_every_n_epochs=2)
    filename = 'checkpoints/Bern_CPMF/' + params_str + '.pkl'

    # Save model params
    with open(filename, 'wb') as f:
        params = {'user_embeddings': model.user_embeddings,
                  'item_embeddings': model.item_embeddings,
                  'user_var': model.user_var,
                  'item_var': model.item_var}
        pickle.dump(params, f, protocol=5)
    
    trial.set_user_attr('filename', filename)
    return MAP

study = run_study(name, objective, n_trials=0)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
best_model_path = study.trials_dataframe().sort_values('value')['user_attrs_filename'].iloc[-1]
with open(best_model_path, 'rb') as f:
    params = pickle.load(f)
    bern_cpmf = init_model()
    bern_cpmf.user_embeddings = params['user_embeddings']
    bern_cpmf.item_embeddings = params['item_embeddings']
    bern_cpmf.user_var = params['user_var']
    bern_cpmf.item_var = params['item_var']

results = test_chap_five(bern_cpmf, data, max_k=10, name='HBR-R', debug=True)
clear_output(wait=True)
# print(results['MAP'].mean(0)[0], results['MAP'].mean(0)[-1])

import copy
hbrY = copy.copy(bern_cpmf)
hbrY.interact = hbrY.interact_return_logit_normal

results = test_chap_five(hbrY, data, max_k=10, name='HBR-Y', debug=True)
clear_output(wait=True)
print(results['MAP'].mean(0)[0], results['MAP'].mean(0)[-1])

best_runs

## Results

In [ ]:
results = {}
for key in os.listdir('results'):
    results[key.replace('.pkl', '').replace('_final', '')] = pickle.load(open(os.path.join('results', key), 'rb'))
order = ['MF-MSE', 'AUR', 'CPMF', 'Neural-AUR', 'HBR', 'HBR-Y']
results = pd.DataFrame([results[key] for key in order], index=order)
results.rename(index={'Neural-AUR': 'Gaussian-MLP', 'CPMF': 'ICPMF', 'HBR': 'HBR-R', 'MF-MSE': 'MF (Baseline)'}, inplace=True)

# Plot aestetics
colors = [c for c in list(TABLEAU_COLORS)]
colors = {k:c for k, c in zip(results.index, colors)}
lines = ['o', 'v', '^', '<', '>', 's', 'p', '+', 'x', '*', 'p']
lines = {k: '-' + l for k, l, in zip(results.index, lines)}

## Baselines
f, ax = plt.subplots(figsize=(10, 4), ncols=2)
for index, row in results.iterrows():
    if index == 'MF (Baseline)':
        ax[0].plot(np.repeat(row['MAP'][-1], 11), '--', color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.repeat(row['Recall'][-1], 11), '--', color=colors[index], label=index, linewidth=3, alpha=0.6)
    else:
        ax[0].plot(row['MAP'].mean(0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(row['Recall'].mean(0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
for line in [0, 1]:
    ax[line].set_xticks(np.arange(11))
    ax[line].set_xticklabels(np.linspace(0, 1, 11).round(2))
    ax[line].set_xlabel(r'$\lambda$', fontsize=20)
ax[0].set_ylabel('MAP@10', fontsize=20)
ax[1].set_ylabel('Recall@10', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
# f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.851, 1.2))
f.tight_layout()
f.savefig('plots/accuracy5.pdf', bbox_inches='tight')


f, ax = plt.subplots(figsize=(10, 4), ncols=2)
for index, row in results.iterrows():
    if index == 'MF (Baseline)':
        print(row['Map relative'])
        ax[0].plot(np.repeat(row['Map relative'][-1] -0.045, 11), '--', color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.repeat(row['Recall relative'][-1] -0.111, 11), '--', color=colors[index], label=index, linewidth=3, alpha=0.6)
    else:
        ax[0].plot(np.nanmean(row['Map relative'], 0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.nanmean(row['Recall relative'], 0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
for line in [0, 1]:
    ax[line].set_xticks(np.arange(0, 11))
    ax[line].set_xticklabels(np.linspace(0, 1, 11).round(2))
    ax[line].set_xlabel(r'$\lambda$', fontsize=20)
ax[0].set_ylabel('MAP Relative@10', fontsize=20)
ax[1].set_ylabel('Recall Relative@10', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
# f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.851, 1.2))
f.tight_layout()
f.savefig('plots/accuracy_relative5.pdf', bbox_inches='tight')


# # Correlation plot
# f, ax = plt.subplots(figsize=(8, 5), ncols=1)
# corr = results[['corr_usup', 'corr_isup']]
# corr.columns=[r'#$R_{u\cdot}$', r'#$R_{\cdot i}$']
# # corr.loc['NEG-ITEM-SUPPORT', r'#$R_{\cdot i}$'] = np.nan
# sns.heatmap(corr.round(3), annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')
# plt.yticks(fontsize=15)
# plt.xticks(rotation=45, fontsize=20)
# plt.tight_layout()
# plt.savefig('plots/corr5.pdf')

# # Ratio
# indexs = [index for index in order if index != 'Baseline']
# f, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
# for index, row in results.iterrows():
#     ax[0].plot(row['MAP'].mean(0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
#     ax[1].plot(row['Recall'].mean(0)[:, -1], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
# ax[0].set_xticks(np.arange(0, 11))
# ax[0].set_xticklabels(np.linspace(0, 1, 11).round(2))
# ax[0].set_xlabel(r'$\lambda$', fontsize=20)
# ax[0].set_ylabel('MAP@10', fontsize=20)
# ax[1].set_xticks(np.arange(0, 11))
# ax[1].set_xticklabels(np.linspace(0, 1, 11).round(2))
# ax[1].set_xlabel(r'$\lambda$', fontsize=20)
# ax[1].set_ylabel('Recall@10', fontsize=20)
# f.tight_layout()
# f.savefig('plots/accuracy_ratio.pdf', bbox_inches="tight")

In [ ]:
f, ax = plt.subplots(nrows=3, ncols=2, figsize=[12, 10])
idx = [(0, 0), (0, 1), (1, 0), (1, 1), (2, 0)]
model = [aur, icpmf, gaussianMLP, bern_cpmf, 0]
label = ['AUR', 'ICPMF', 'Gaussian-MLP', 'HBR-R', 'HBR-Y']
for idx, model, label in zip(idx, model, label):
    if label == 'HBR-Y':
        mu, var = rand_preds[0], rand_preds[1]
        constant =  3 / torch.pi**2
        denom = np.sqrt(1 + constant * var)
        mu_logistic_normal = 1 / (1 + np.exp(-mu / denom))
        var_logistic_normal = mu_logistic_normal * (1 - mu_logistic_normal) * (1 - 1 / denom)
        rand_preds = [mu_logistic_normal, var_logistic_normal]
        ax[idx].set_xlabel(r'Relevance: $E[P_{ui}]$', fontsize=20)
        ax[idx].set_ylabel(r'Uncertainty: $Var[P_{ui}]$', fontsize=20)
        idx_valid = np.logical_and(rand_preds[1] > -5, rand_preds[1] < 0.1)
    else:
        rand_preds = model.predict(data.rand['users'][:5000], data.rand['items'][:5000])
        ax[idx].set_xlabel(r'Relevance: $\mu_{ui}$', fontsize=20)
        ax[idx].set_ylabel(r'Uncertainty: $\sigma^2_{ui}$', fontsize=20)
        idx_valid = np.logical_and(rand_preds[1] > -5, rand_preds[1] < 5)
    if not 'HBR' in label:
        rand_preds = rand_preds[0], np.exp(rand_preds[1])
    ax[idx].plot(rand_preds[0][idx_valid], rand_preds[1][idx_valid], 'o')
    ax[idx].annotate(label, fontsize=20, xy=(0.55, 0.90), xycoords='axes fraction')
    b, a = np.polyfit(rand_preds[0][idx_valid], rand_preds[1][idx_valid], deg=1)
    xseq = np.linspace(rand_preds[0][idx_valid].min(), rand_preds[0][idx_valid].max(), num=100)
    ax[idx].plot(xseq, a + b * xseq, color="k", lw=2.5)


f.delaxes(ax[2, 1])
f.tight_layout()
f.savefig('plots/relevance_uncertainty5.pdf')

# Not used

This is some extra stuff tested during development that were not used in our papers.

## DoubleMF

In [ ]:
name = 'AUR-prior'
def init_model(**kwargs):
    return JointDoubleMF(data.n_user, data.n_item, embedding_dim=128, embedding_dim_var=128, beta=1/2, ratio=0.9, loss='AUR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-5, 1e-5, log=True),
              'gamma': trial.suggest_float('gamma', 1e-5, 1e-4, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-5, 1e-4, log=True),
              'n_negatives': trial.suggest_int('neg', 1, 1)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)

    # Rename model file
    os.rename('checkpoints/' + name + '/last.ckpt', 'checkpoints/' + name + '/' + params_str + '_last.ckpt')
    
    return MAP

study = run_study(name, objective, n_trials=0)
best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_chap_five(model, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

In [ ]:
study.trials_dataframe().sort_values('value')[::-1]

In [ ]:
name = 'CPMF'
def init_model(**kwargs):
    return JointDoubleMF(data.n_user, data.n_item, embedding_dim=128, embedding_dim_var=1, beta=1/2, ratio=0.1, loss='AUR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-5, 1e-4, log=True),
              'gamma': trial.suggest_float('gamma', 1e-3, 1e-1, log=True),
              'weight_decay': trial.suggest_float('wd', 1e-6, 1e-3, log=True),
              'n_negatives': trial.suggest_int('neg', 1, 50)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    print(MAP, path, train_likelihood)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)

    # Rename model file
    os.rename('checkpoints/' + name + '/last.ckpt', 'checkpoints/' + name + '/' + params_str + '_last.ckpt')
    
    return MAP

study = run_study(name, objective, n_trials=4)
best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_chap_five(model, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

## Neural

In [ ]:
name = 'Neural-AUR-large'
def init_model(**kwargs):
    return GaussianMLP(data.n_user, data.n_item, embedding_dim=128, beta=1/2, ratio=0.1, loss='AUR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'lr': trial.suggest_float('lr', 1e-3, 1e-3, log=True),
              'gamma': trial.suggest_float('gamma', 1e-2, 1e-1, log=True),
              'dropout': trial.suggest_float('dropout', 0, 0.2),
              'n_negatives': trial.suggest_int('neg', 1, 1)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=1)
best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
model.dropl.eval()
results = test_chap_five(model, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

In [ ]:
study.trials_dataframe().sort_values('value')[::-1]

## Sequential

In [ ]:
name = 'AUR-S'
def init_model(**kwargs):
    return SequentialDoubleMF(baseline=mfmse, embedding_dim_var=128, beta=1/2, loss='AUR', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'n_negatives': trial.suggest_int('neg', 20, 50),
              'gamma': trial.suggest_float('gamma', 1e-2, 1e-1, log=True)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    recall, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return recall

study = run_study(name, objective, n_trials=20)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_uncertain(model, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

In [ ]:
name = 'GBR-S'
def init_model(**kwargs):
    return SequentialDoubleMF(baseline=mfpoint, embedding_dim_var=128, loss='Pointwise', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'n_negatives': trial.suggest_int('neg', 10, 40),
              'gamma': trial.suggest_float('gamma', 1e-8, 1e-8, log=True)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    recall, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return recall

study = run_study(name, objective, n_trials=2)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_uncertain(model, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

In [ ]:
name = 'GPR-S'
def init_model(**kwargs):
    return SequentialDoubleMF(baseline=mfpair, embedding_dim_var=128, loss='Pairwise', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'gamma': trial.suggest_float('gamma', 1e-6, 1e-2, log=True)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size)
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    recall, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return recall

study = run_study(name, objective, n_trials=5)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_uncertain(model, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

## Joint

In [ ]:
name = 'AUR-J'
def init_model(**kwargs):
    return JointDoubleMF(data.n_user, data.n_item, embedding_dim=128, embedding_dim_var=128, loss='AUR', beta=1/2, **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'weight_decay': trial.suggest_float('wd', 1e-5, 1e-4, log=True),
              'n_negatives': trial.suggest_int('neg', 20, 50),
              'gamma': trial.suggest_float('gamma', 1e-2, 1e-1, log=True)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    recall, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return recall

study = run_study(name, objective, n_trials=1)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
# results = test_uncertain(model, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

In [ ]:
name = 'GBR-J'
def init_model(**kwargs):
    return JointDoubleMF(data.n_user, data.n_item, embedding_dim=128, embedding_dim_var=128, loss='Pointwise', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'weight_decay': trial.suggest_float('wd', 1e-5, 1e-2, log=True),
              'n_negatives': trial.suggest_int('neg', 10, 40),
              'gamma': trial.suggest_float('gamma', 1e-5, 1e-2, log=True)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    recall, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return recall

study = run_study(name, objective, n_trials=10)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_uncertain(model, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

In [ ]:
name = 'GPR-J'
def init_model(**kwargs):
    return JointDoubleMF(data.n_user, data.n_item, embedding_dim=128, embedding_dim_var=128, loss='Pairwise', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'weight_decay': trial.suggest_float('wd', 1e-5, 1e-2, log=True),
              'gamma': trial.suggest_float('gamma', 1e-6, 1e-2, log=True)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size)
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    recall, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return recall

study = run_study(name, objective, n_trials=5)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_uncertain(model, data, max_k=10, name=name)

clear_output(wait=True)
best_runs

## Results

In [ ]:
# Load results
results = {}
for key in os.listdir('results'):
    results[key.replace('.pkl', '').replace('_', ' ')] = pickle.load(open(os.path.join('results', key), 'rb'))
order = ['MF-BPR', 'MF-BCE', 'MF-Pointwise', 'MF-Pairwise', 'MF-MSE', 
         'AUR-J', 'GPR-J', 'GBR-J',
         'AUR-S', 'GPR-S', 'GBR-S']
drop = ['MF-BPR', 'MF-BCE', 'MF-Pointwise', 'MF-Pairwise', 'MF-MSE']
results = pd.DataFrame([results[key] for key in order], index=order)

# Plot aestetics
colors = [c for c in list(TABLEAU_COLORS)] + ['k', 'b', 'g', 'r', 'm']
colors = {k:c for k, c in zip(results.index, colors)}
lines = ['o', 'v', '^', '<', '>', 's', 'p', '+', 'x', '*', 'p']
lines = {k: '-' + l for k, l, in zip(results.index, lines)}

# Results
results = pd.DataFrame([results[key] for key in order], index=order)
results[['FCP', 'URI', 'UAC']].T

In [ ]:
# Correlation plot
corr = results.drop(index=drop)[['corr_usup', 'corr_isup']]
corr.columns=[r'#$R_{u\cdot}$', r'#$R_{\cdot i}$']
sns.heatmap(corr.round(3), annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')
plt.xticks(rotation=45, fontsize=15)
plt.tight_layout()
plt.savefig('plots/corr.pdf')

In [ ]:
## Top-K accuracy metrics
f, ax = plt.subplots(figsize=(10, 5), ncols=2)
for index, row in results.drop(index=['MF-BPR', 'MF-MSE', 'MF-Pointwise', 'MF-Pairwise']).iterrows():
    try:
        ax[0].plot(np.arange(1, 11), row['MAP2'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.arange(1, 11), row['Recall2'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
    except:
        ax[0].plot(np.arange(1, 11), row['MAP'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.arange(1, 11), row['Recall'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax[0].set_xticks(np.arange(1, 11))
ax[0].set_xlabel('n', fontsize=20)
ax[0].set_ylabel('MAP@n', fontsize=20)
ax[1].set_xticks(np.arange(1, 11))
ax[1].set_xlabel('n', fontsize=20)
ax[1].set_ylabel('Recall@n', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=4, bbox_to_anchor=(0.86, 1.12))
f.tight_layout()
f.savefig('plots/accuracy.pdf')
results.drop(index=['MF-BPR', 'MF-BCE'])[['MAP', 'MAP2', 'Recall', 'Recall2']].transform(lambda x: x.str[-1])

In [ ]:
## MAP vs Uncertainty and vs UserUnceratinty
f, ax = plt.subplots(figsize=(6, 5))
x = np.arange(10) + 1
for index, row in results.drop(index=drop).iterrows():
    ax.plot(x, row['MAP-Uncertainty'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax.set(xticks = x)
ax.set_xlabel('User uncertainty bin', fontsize=25)
ax.set_ylabel('MAP@10', fontsize=20)
handles, labels = ax.get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.94, 1.10))
f.tight_layout()
f.savefig('plots/MAP-Uncertainty.pdf')

In [ ]:
## MAP and Uncertainty vs Profile size
f, ax = plt.subplots(figsize=(6, 5))
for index, row in results.drop(index=drop).iterrows():
    ax.plot(x, row['MAP-ProfSize'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax.set(xticks = x)
ax.set_xlabel('Profile size bin', fontsize=25)
ax.set_ylabel('MAP@10', fontsize=20)
handles, labels = ax.get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.94, 1.10))
f.tight_layout()
f.savefig('plots/MAP-ProfSize.pdf')

In [ ]:
name = 'MLP-GBR'
def init_model(**kwargs):
    return GaussianMLP(data.n_user, data.n_item, embedding_dim=128, loss='Pointwise', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'dropout': trial.suggest_float('dropout', 0, 0.2),
              'n_negatives': trial.suggest_int('neg', 5, 50)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=5)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_vanilla(model, data, max_k=10, name=name)

clear_output(wait=True)
print(results['MAP'])
best_runs

In [ ]:
name = 'MLP-GBR'
def init_model(**kwargs):
    return GaussianMLP(data.n_user, data.n_item, embedding_dim=128, loss='Pointwise', **kwargs)

def objective(trial):
    
    # Parameter setup
    params = {'dropout': trial.suggest_float('dropout', 0, 0.2),
              'n_negatives': trial.suggest_int('neg', 5, 50)}
    params_str = '-'.join(f'{key}={value}' for key, value in params.items())
    data.batch_size = int(base_batch_size / (params['n_negatives'] + 1))
    print(params, data.batch_size)

    # Train
    model = init_model(**params)
    MAP, path, train_likelihood = train(model, data, path='checkpoints/' + name, name=params_str, patience=patience)
    trial.set_user_attr('filename', path)
    trial.set_user_attr('mean_loss', train_likelihood)
    return MAP

study = run_study(name, objective, n_trials=5)
best_runs = study.trials_dataframe().sort_values('value')[::-1]
model = init_model().load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_vanilla(model, data, max_k=10, name=name)

clear_output(wait=True)
print(results['MAP'])
best_runs

## Ensemble

## MLP

In [ ]:
name = 'MLP'
def init_model(**kwargs):
    return MLP(data.n_user, data.n_item, **kwargs)
base_conf = {'embedding_dim': 10, 'lr': 0, 'n_negatives': 0}

def objective(trial):
    
    # Parameter setup
    batch_size = trial.suggest_int('bs', 256, 256)
    params = {'embedding_dim': trial.suggest_int('dim', 64, 64),
              'lr': trial.suggest_float('lr', 1e-4, 1e-2),
              'n_negatives': trial.suggest_int('neg', 2, 20),
              'dropout': trial.suggest_float('layers', 0, 0.2)}
    params_str = f'bs = {batch_size}-' + '-'.join(f'{key}={value}' for key, value in params.items())

    # Train
    data.batch_size = batch_size
    model = init_model(**params)
    MAP, path = train(model, data, path='checkpoints/' + name, name=params_str)
    trial.set_user_attr('filename', path)
    return MAP

study = run_study(name, objective, n_trials=trials)
mlp, runs = load(init_model(**base_conf), study)
# results = test_vanilla(mlp, data, max_k=10, name=name)

clear_output(wait=True)
# print(results)
runs[:5]

## Results

In [ ]:
# Load results
results = {}
for key in os.listdir('results'):
    results[key.replace('.pkl', '').replace('_', ' ')] = pickle.load(open(os.path.join('results', key), 'rb'))
order = ['MF-BPR', 'MF-BCE', 'MF-Pointwise', 'MF-Pairwise', 'MF-MSE', 
         'AUR', 'Pointwise', 'Pairwise']
drop = ['MF-BPR', 'MF-BCE', 'MF-Pointwise', 'MF-Pairwise', 'MF-MSE']
results = pd.DataFrame([results[key] for key in order], index=order)

# Plot aestetics
colors = [c for c in list(TABLEAU_COLORS)] + ['k', 'b', 'g', 'r', 'm']
colors = {k:c for k, c in zip(results.index, colors)}
lines = ['o', 'v', '^', '<', '>', 's', 'p', '+', 'x', '*', 'p']
lines = {k: '-' + l for k, l, in zip(results.index, lines)}

# Results
results[['FCP', 'URI', 'UAC']].T

In [ ]:
# Correlation plot
corr = results.drop(index=drop)[['corr_usup', 'corr_isup']]
corr.columns=[r'#$R_{u\cdot}$', r'#$R_{\cdot i}$']
sns.heatmap(corr.round(3), annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')
plt.xticks(rotation=45, fontsize=15)
plt.tight_layout()
plt.savefig('plots/corr.pdf')

In [ ]:
## Top-K accuracy metrics
f, ax = plt.subplots(figsize=(10, 5), ncols=2)
for index, row in results.iterrows():
    try:
        ax[0].plot(np.arange(1, 11), row['MAP2'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.arange(1, 11), row['Recall2'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
    except:
        ax[0].plot(np.arange(1, 11), row['MAP'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
        ax[1].plot(np.arange(1, 11), row['Recall'], lines[index], color=colors[index], label=index, linewidth=3, alpha=0.6)
ax[0].set_xticks(np.arange(1, 11))
ax[0].set_xlabel('n', fontsize=20)
ax[0].set_ylabel('MAP@n', fontsize=20)
ax[1].set_xticks(np.arange(1, 11))
ax[1].set_xlabel('n', fontsize=20)
ax[1].set_ylabel('Recall@n', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.9, 1.3))
f.tight_layout()
f.savefig('plots/accuracy.pdf', bbox_inches='tight')

table = results.drop(index=drop)[['MAP', 'MAP2', 'Recall', 'Recall2']].transform(lambda x: x.str[-1])
table.columns = ['MAP', 'UA MAP', 'Recall', 'UA Recall']
table['MAP-Improvement'] = (((table['UA MAP'] / table['MAP']) - 1) * 100).round(3).astype(str).add('%')
table['Recall-Improvement'] = (((table['UA Recall'] / table['Recall']) - 1) * 100).round(3).astype(str).add('%')
table.to_csv('plots/improvements.csv')
table

In [ ]:
# Load results
results = {}
for key in os.listdir('results'):
    results[key.replace('.pkl', '').replace('_', ' ')] = pickle.load(open(os.path.join('results', key), 'rb'))
order = ['MF', 'MF-ENSEMBLE', 'MLP', 'MLP-ENSEMBLE', 'MCDropout', 'BayesianMLP', 'MF-NIS' , 'MLP-NIS', 'MF-NUS' , 'MLP-NUS']
results = pd.DataFrame([results[key] for key in order], index=order)

# Plot aestetics
colors = [c for c in list(TABLEAU_COLORS)] + ['k', 'b', 'g', 'r']
colors = {k:c for k, c in zip(results.index, colors)}
lines = ['o', 'v', '^', '<', '>', 's', 'p', '+', 'x', '*']
lines = {k: '-' + l for k, l, in zip(results, lines)}

# Results
results[['FCP', 'URI', 'UAC']]

In [ ]:
## Top-K accuracy metrics
f, ax = plt.subplots(figsize=(10, 5), ncols=2)
for index, row in results.drop(['MF-NIS', 'MF-NUS', 'MLP-NUS', 'MLP-NIS']).iterrows():
    ax[0].plot(np.arange(1, 11), row['MAP'], '-', color=colors[index], label=index, linewidth=5, alpha=0.6)
    ax[1].plot(np.arange(1, 11), row['Recall'], '-', color=colors[index], label=index, linewidth=5, alpha=0.6)
ax[0].set_xticks(np.arange(1, 11))
ax[0].set_xlabel('n', fontsize=20)
ax[0].set_ylabel('MAP@n', fontsize=20)
ax[1].set_xticks(np.arange(1, 11))
ax[1].set_xlabel('n', fontsize=20)
ax[1].set_ylabel('Recall@n', fontsize=20)
handles, labels = ax[0].get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.85, 1.1))
f.tight_layout()
f.savefig('plots/accuracy.pdf')

In [ ]:
## MAP vs Uncertainty
f, ax = plt.subplots(figsize=(6, 5))
x = np.arange(10) + 1
for index, row in results.drop(['MF', 'MLP']).iterrows():
    ax.plot(x, row['MAP-Uncertainty'], '-', color=colors[index], label=index, linewidth=5, alpha=0.6)
ax.set(xticks = x)
ax.set_xlabel('Uncertainty bin', fontsize=25)
ax.set_ylabel('MAP@10', fontsize=20)
handles, labels = ax.get_legend_handles_labels()
f.legend(handles, labels, fontsize=15, ncol=3, bbox_to_anchor=(0.85, 1.1))
f.tight_layout()
f.savefig('plots/MAP-Uncertainty.pdf')

In [ ]:
## MAP vs Coverage
f, ax = plt.subplots(figsize=(6, 5))
x = np.linspace(-3, 3, 11)
for index, row in results.drop(['MF', 'MLP']).iterrows():
    ax.plot(x, row['unc_MAP'], '-', color=colors[index], label=index, linewidth=5, alpha=0.6)
ax.set(xticks = x, xticklabels = [str(round(x, 1)) for x in x])
ax.set_xlabel(r'$\lambda$', fontsize=25)
ax.set_ylabel('MAP@10', fontsize=20)
ax.legend(fontsize=15)
f.tight_layout()
f.savefig('plots/accuracy_lambda.pdf')

In [ ]:
idx = ['MCDropout', 'BayesianMLP']
f, ax = plt.subplots(figsize=(12, 5), ncols=len(idx))
for i, model in enumerate(idx):
    ax[i].hist(results.loc[model]['norm_unc'][0], bins='auto', alpha=0.6, density=True, label='Hits')
    ax[i].hist(results.loc[model]['norm_unc'][1], bins='auto', alpha=0.4, density=True, label='Non-hits')
    ax[i].legend(fontsize=15)
    ax[i].set_xlabel('Normalized uncertainty', fontsize=20)
    ax[i].set_ylabel('Density', fontsize=20)
    ax[i].set_title(model)
f.tight_layout()
f.savefig('plots/normalized_uncertainty.pdf')